# NFL Player Clustering By Position

## Data Pre-Processing

In [143]:
import pandas as pd

In [144]:
passing_basic_stats = pd.read_csv("C:/Users/jakel/OneDrive/Desktop/GitHub/NFL-Player-Clustering/pro-football-reference-data/Passing-BasicStats.csv")
passing_basic_stats.head()

,Rk,Player,Age,Team,Pos,G,GS,QBrec,Cmp,Att,...,QBR,Sk,Yds.1,Sk%,NY/A,ANY/A,4QC,GWD,Awards,Player-additional
0,1.0,Joe Burrow,28.0,CIN,QB,17.0,17.0,9-8-0,460.0,652.0,...,74.7,48.0,278.0,6.86,6.63,7.28,1.0,2.0,PBAP MVP-4AP OPoY-5AP CPoY-1,BurrJo01
1,2.0,Jared Goff,30.0,DET,QB,17.0,17.0,15-2-0,390.0,539.0,...,68.4,31.0,234.0,5.44,7.71,8.06,4.0,4.0,PBAP MVP-5AP OPoY-9,GoffJa00
2,3.0,Baker Mayfield,29.0,TAM,QB,17.0,17.0,10-7-0,407.0,570.0,...,61.6,40.0,248.0,6.56,6.97,7.13,2.0,2.0,PBAP MVP-11AP OPoY-11,MayfBa00
3,4.0,Geno Smith,34.0,SEA,QB,17.0,17.0,10-7-0,407.0,578.0,...,53.8,50.0,338.0,7.96,6.34,5.93,4.0,4.0,NaN,SmitGe00
4,5.0,Sam Darnold,27.0,MIN,QB,17.0,17.0,14-3-0,361.0,545.0,...,60.4,48.0,335.0,8.09,6.72,6.99,3.0,5.0,PBAP MVP-10AP CPoY-3,DarnSa00


In [145]:
rushing_basic_stats = pd.read_csv("C:/Users/jakel/OneDrive/Desktop/GitHub/NFL-Player-Clustering/pro-football-reference-data/Rushing-BasicStats.csv")
rushing_basic_stats.head()

,Rk,Player,Age,Team,Pos,G,GS,Att,Yds,TD,1D,Succ%,Lng,Y/A,Y/G,A/G,Fmb,Awards,-9999
0,1.0,Saquon Barkley,27.0,PHI,RB,16.0,16.0,345.0,2005.0,13.0,82.0,52.5,72.0,5.8,125.3,21.6,2.0,PBAP-1AP MVP-3AP OPoY-1AP CPoY-14,BarkSa00
1,2.0,Derrick Henry,30.0,BAL,RB,17.0,17.0,325.0,1921.0,16.0,94.0,58.8,87.0,5.9,113.0,19.1,3.0,PBAP-2AP OPoY-4,HenrDe00
2,3.0,Kyren Williams,24.0,LAR,RB,16.0,16.0,316.0,1299.0,14.0,85.0,50.9,30.0,4.1,81.2,19.8,5.0,AP OPoY-11,WillKy02
3,4.0,Bijan Robinson,22.0,ATL,RB,17.0,17.0,304.0,1456.0,14.0,82.0,60.2,37.0,4.8,85.6,17.9,1.0,PB,RobiBi01
4,5.0,Jonathan Taylor,25.0,IND,RB,14.0,13.0,303.0,1431.0,11.0,71.0,45.9,70.0,4.7,102.2,21.6,4.0,PBAP CPoY-19,TaylJo02


In [146]:
receiving_basic_stats = pd.read_csv("C:/Users/jakel/OneDrive/Desktop/GitHub/NFL-Player-Clustering/pro-football-reference-data/Receiving-BasicStats.csv")
receiving_basic_stats.head()

,Rk,Player,Age,Team,Pos,G,GS,Tgt,Rec,Yds,...,1D,Succ%,Lng,R/G,Y/G,Ctch%,Y/Tgt,Fmb,Awards,-9999
0,1.0,Ja'Marr Chase,24.0,CIN,WR,17.0,16.0,175.0,127.0,1708.0,...,75.0,62.3,70.0,7.5,100.5,72.6,9.8,0.0,PBAP-1AP MVP-8AP OPoY-3,ChasJa00
1,2.0,Amon-Ra St. Brown,25.0,DET,WR,17.0,17.0,141.0,115.0,1263.0,...,73.0,70.2,66.0,6.8,74.3,81.6,9.0,1.0,PBAP-1AP OPoY-11,StxxAm00
2,3.0,Brock Bowers,22.0,LVR,TE,17.0,16.0,153.0,112.0,1194.0,...,61.0,60.8,57.0,6.6,70.2,73.2,7.8,0.0,PBAP-1AP ORoY-2,BoweBr01
3,4.0,Trey McBride,25.0,ARI,TE,16.0,16.0,147.0,111.0,1146.0,...,63.0,63.3,37.0,6.9,71.6,75.5,7.8,0.0,PB,McBrTr01
4,5.0,Malik Nabers,21.0,NYG,WR,15.0,13.0,170.0,109.0,1204.0,...,55.0,51.2,59.0,7.3,80.3,64.1,7.1,1.0,PBAP ORoY-5,NabeMa00


# Processing QB Stats:
## Filter out unnecessary cols

In [147]:
passing_basic_stats.columns

Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'QBrec', 'Cmp', 'Att',
       'Cmp%', 'Yds', 'TD', 'TD%', 'Int', 'Int%', '1D', 'Succ%', 'Lng', 'Y/A',
       'AY/A', 'Y/C', 'Y/G', 'Rate', 'QBR', 'Sk', 'Yds.1', 'Sk%', 'NY/A',
       'ANY/A', '4QC', 'GWD', 'Awards', 'Player-additional'],
      dtype='object')

In [148]:
def get_wins(record):
    if isinstance(record, str) and '-' in record:
        split_rec = record.split('-')
        return int(split_rec[0])
    return 0

In [149]:
passing_basic_stats.drop(columns=['Rk', 'Cmp', 'Att', 'Sk', 'Yds.1', 'Sk%', 'Awards', 'Player-additional'], inplace=True)
# (Drop sack related stats b/c I'm a Bears fan and those are NOT QB Stats :P )
passing_basic_stats = passing_basic_stats[passing_basic_stats['Pos'] == 'QB']
passing_basic_stats = passing_basic_stats[passing_basic_stats['GS'] >= 9]
passing_basic_stats['QB-Wins'] = passing_basic_stats['QBrec'].apply(get_wins)
passing_basic_stats.drop(columns=['QBrec', 'Pos'], inplace=True)
passing_basic_stats

,Player,Age,Team,G,GS,Cmp%,Yds,TD,TD%,Int,...,AY/A,Y/C,Y/G,Rate,QBR,NY/A,ANY/A,4QC,GWD,QB-Wins
0,Joe Burrow,28.0,CIN,17.0,17.0,70.6,4918.0,43.0,6.6,9.0,...,8.24,10.7,289.3,108.5,74.7,6.63,7.28,1.0,2.0,9
1,Jared Goff,30.0,DET,17.0,17.0,72.4,4629.0,37.0,6.9,12.0,...,8.96,11.9,272.3,111.8,68.4,7.71,8.06,4.0,4.0,15
2,Baker Mayfield,29.0,TAM,17.0,17.0,71.4,4500.0,41.0,7.2,16.0,...,8.07,11.1,264.7,106.8,61.6,6.97,7.13,2.0,2.0,10
3,Geno Smith,34.0,SEA,17.0,17.0,70.4,4320.0,21.0,3.6,15.0,...,7.03,10.6,254.1,93.2,53.8,6.34,5.93,4.0,4.0,10
4,Sam Darnold,27.0,MIN,17.0,17.0,66.2,4319.0,35.0,6.4,12.0,...,8.22,12.0,254.1,102.5,60.4,6.72,6.99,3.0,5.0,14
5,Lamar Jackson,27.0,BAL,17.0,17.0,66.7,4172.0,41.0,8.6,4.0,...,10.15,13.2,245.4,119.6,77.3,8.09,9.38,2.0,2.0,12
6,Patrick Mahomes,29.0,KAN,16.0,16.0,67.5,3928.0,26.0,4.5,11.0,...,6.80,10.0,245.5,93.5,67.6,5.98,6.02,5.0,7.0,15
7,Aaron Rodgers,41.0,NYJ,17.0,17.0,63.0,3897.0,28.0,4.8,11.0,...,6.78,10.6,229.2,90.5,48.0,5.76,5.87,2.0,3.0,5
8,Justin Herbert,26.0,LAC,17.0,17.0,65.9,3870.0,23.0,4.6,3.0,...,8.32,11.7,227.6,101.7,64.8,6.65,7.25,1.0,2.0,11
9,Brock Purdy,25.0,SFO,15.0,15.0,65.9,3864.0,20.0,4.4,12.0,...,8.18,12.9,257.6,96.1,67.9,7.63,7.34,1.0,1.0,6


## Isolate the columns into numeric and non-numeric features for PCA:

In [150]:
NFL_QB_Identifiers = passing_basic_stats[['Player', 'Team']]
NFL_QB_Features = passing_basic_stats.drop(columns=['Player', 'Team'])

NFL_QB_Identifiers.to_csv("C:/Users/jakel/OneDrive/Desktop/GitHub/NFL-Player-Clustering/filtered-data/NFL-QB-Identifiers.csv")
NFL_QB_Features.to_csv("C:/Users/jakel/OneDrive/Desktop/GitHub/NFL-Player-Clustering/filtered-data/NFL-QB-Features.csv")